# **第X章：使用 cuNumeric 进行分布式计算**

## **引言：**

cuNumeric 是一个与 NumPy 兼容的库，专为大规模计算而设计。它旨在提供一个分布式且加速的 NumPy API 替代方案。该库是 Legate 项目的一部分，该项目利用数据中心编程模型 Legion 来提供可扩展/易用的 Python 替代方案，通过利用 GPU 加速和分布式计算。cuNumeric 的目标是替代广泛使用的 Python 库 NumPy。Legate 和类似 cuNumeric 的库的目标是使高性能计算大众化，让程序员能够利用大型 CPU 和 GPU 集群的力量，同时仍然能够在台式机或笔记本电脑上无缝运行相同的代码！

**cuNumeric 与 NumPy 的比较：**

cuNumeric 设计时考虑了 NumPy 的所有功能，使 NumPy 用户能够轻松使用 cuNumeric。但是，它们之间存在几个关键差异：

* 可扩展性：cuNumeric 能够在大型 CPU/GPU 集群上运行，随着 AI 的不断发展，它更适合处理更大规模的数据处理。
* GPU 加速：由于 cuNumeric 利用 GPU 加速，与仅限于 CPU 操作的 NumPy 相比，它能提供显著的性能提升。cuNumeric 使处理大量数据集变得更加容易。
* 分布式计算：cuNumeric 支持分布式计算，允许将计算分散到各个节点上，使用户能够更有效地处理不断增长的问题规模，与单独使用 NumPy 相比，显著减少了总计算时间。

## **硬件要求：**

建议配置（单节点设置）：

* GPU：NVIDIA V100 或更新型号
* CPU：AMD EPYC、Intel Xeon（支持多核，至少16核）
* 系统内存：64 GB 或更多（取决于数据集大小）

其他选项：可在 Perlmutter 等 HPC 集群上运行！（[Perlmutter](https://docs.nersc.gov/systems/perlmutter/architecture/) 是位于国家能源研究科学计算中心（NERSC）的强大超级计算机）

## **快速安装指南：**

在 Linux 服务器上安装 CUDA，请按以下步骤操作：

1. 访问 [CUDA Toolkit 下载页面](https://developer.nvidia.com/cuda-downloads)
2. 选择您的操作系统、架构、发行版和版本
3. 选择安装程序类型（如 runfile 或 deb）
4. 按照提供的安装说明进行操作

在 Linux 机器上安装 Conda，请在终端中输入以下命令：

In [ ]:
mkdir -p ~/miniforge3 
wget https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh -O ~/miniforge3/miniforge.sh
bash ~/miniforge3/miniforge.sh -b -u -p ~/miniforge3 
rm -rf ~/miniforge3/miniforge.sh 
~/miniforge3/bin/conda init bash
~/miniforge3/bin/conda init zsh
source ~/.bashrc

不同操作系统的安装说明：[Conda](https://docs.anaconda.com/miniconda/)

接下来让我们通过在终端中输入以下命令来安装 Legate 和 cuNumeric：

In [ ]:
conda create -n legate -c conda-forge -c legate cunumeric

**普通 Linux 系统：**

In [ ]:
conda activate legate

**Perlmutter（多节点使用）：**

创建交互式会话：
示例：

In [ ]:
salloc --nodes 1 --qos interactive --time 01:30:00 --constraint gpu_ss11 --gpus 2 --account=<group-id> --mem=200G

<上述示例创建了一个交互式会话，包含1个节点，2个 A100 GPU（gpu_ss11），200GB 内存，运行时间为1小时30分钟>

In [ ]:
conda create -n legate -c conda-forge -c legate/label/gex-experimental cunumeric realm-gex-wrapper legate-mpi-wrapper

如果需要更新任何依赖项，请继续更新。

In [ ]:
conda activate legate

要使用多节点，请激活以下包装器：

In [ ]:
/global/homes/<intial>/<username>/miniconda3/envs/legate/gex-wrapper/build-gex-wrapper.sh

/global/homes/<intial>/<username>/miniconda3/envs/legate/mpi-wrapper/build-gex-wrapper.sh

如果需要重新激活环境，请重新运行 conda activate legate。

### **示例运行：**

创建一个 main.py 文件，并输入以下代码：

In [ ]:
import cunumeric as np
from legate.timing import time
# 定义向量大小
start_time = time()
size = 100000000 

# 生成两个指定大小的随机向量
vector1 = np.random.rand(size)
vector2 = np.random.rand(size)

# 使用 cuNumeric 计算点积
dot_product = np.dot(vector1, vector2)

# 打印结果
end_time = time()
elapsed_time = (end_time - start_time)/1000

print("点积结果:", dot_product)
print(f"点积计算耗时 {elapsed_time:.4f} 毫秒")

要运行上述代码，请遵循以下结构：

legate --<处理器类型> --<CLI选项> ./main.py <main.py选项>
#例如，让我们使用：

In [ ]:
legate --gpus 4 --sysmem 2000 --fbmem 12000 --eager-alloc-percentage 10 ./main.py

输出：

如果您能在本地运行，您可能可以使用 matplotlib 绘制图像。您只需要在顶部取消注释库访问行，并取消注释绘制图像的部分。如果正确完成，您应该得到一个如下所示的图像：

![CuImageEx2.png](attachment:328fe50f-9430-41ba-9f4a-78dae0b8677e.png)

# **资源分配/使用 Legate：**

如前所述，cuNumeric 是 Legate 项目的一部分。
Legate 将用于执行您的程序。它允许您指定成功执行应用程序所需的资源。
有3种不同类型的处理器（任务变体）：CPU、GPU 和 OMP。（OMP 变体使用 OpenMP 线程来并行化程序）
除了指定变体外，您还必须为每个处理器上的程序指定所需的内存量：

可用参数：

In [ ]:
- - nodes: 程序要使用的节点数

- - cpus: 程序要使用的 CPU 数量
- - gpus: 程序要使用的 GPU 数量

- - omps: 创建的 OMP 进程数
- - ompthreads: OMP 进程中的线程数

- - sysmem: 系统内存（MB）
- - fbmem: 每个 GPU 的帧缓冲内存（MB）
- - eager-alloc-percentage: 为急切分配保留的百分比

运行程序的模板：

In [ ]:
#CPU 示例

legate --cpus 8 --sysmem 40000 --eager-alloc-percentage 10 ./main.py <main.py选项>

#GPU 示例

legate --gpus 2 --fbmem 40000 --eager-alloc-percentage 10 ./main.py <main.py选项>

#OMP 示例

legate --omps 1 --ompthreads 4 --sysmem 40000 --eager-alloc-percentage 10 ./main.py <main.py选项>

有关命令行界面选项的更多信息，请参阅 Legate CLI 选项

# **最佳实践：**

从 NumPy 开始：如果您熟悉 NumPy，可以像平常一样用 NumPy 编写代码，然后将 'import numpy as np 替换为 import cunumeric as np'。

In [ ]:
import numpy as np

变为

In [ ]:
import cunumeric as np

**三大要点：**

1. *使用 cuNumeric 数组而不是原生列表*
   使用数组时，不要使用原生 Python 数据结构如列表或元组。从这些结构创建 cuNumeric 数组，并使用基于数组的函数对其进行操作。

In [ ]:
# 不推荐：使用列表和其他原生 Python 数据结构进行大规模计算
x = [1, 2, 3]
y = []
for val in x:
    y.append(val + 2)

# 推荐：创建 cuNumeric 数组并使用基于数组的操作
y = np.array(x)
y = x + 2

2. *避免使用索引的循环；利用基于数组的操作*
   如果需要设置/更新数组的某个组件，请用基于数组的实现替换显式循环

In [ ]:
# 示例 1
# x 和 y 是三维数组

# 不推荐：朴素的逐元素实现
for i in range(ny):
    for j in range(nx):
        x[0, j, i] = y[3, j, i]

# 推荐：基于数组的实现
x[0] = y[3]

# 示例 2

# x 和 y 是二维数组，我们需要根据 y 是否满足条件来更新 x

# 不推荐：朴素的逐元素实现
for i in range(ny):
    for j in range(nx):
        if (y[j, i] < tol):
            x[j, i] = const
        else
            x[j, i] = 1.0 - const

# 推荐：基于数组的实现
cond = y < tol
x[cond] = const
x[~cond] = 1.0 - const

3. *使用布尔掩码，避免高级索引*
   使用布尔掩码索引数组将比使用带索引的数组更快

In [ ]:
import cunumeric as np

# 不推荐：不要使用 nonzero 获取索引
indices = np.nonzero(h < 0)
x[indices] = y[indices]

# 推荐：使用布尔掩码更新数组
cond = h < 0
x[cond] = y[cond]

更多最佳实践，请访问 cuNumeric [最佳实践](https://docs.nvidia.com/cunumeric/24.06/user/practices.html)！